In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from env import user, password, host
import warnings
warnings.filterwarnings('ignore')

def acquire_zillow():
    if os.path.exists('zillow_2017.csv'):
        return pd.read_csv('zillow_2017.csv', index_col=0)
    else:
        ''' Acquire data from Zillow using env imports and rename columns'''

        url = f"mysql+pymysql://{user}:{password}@{host}/zillow"

        query = """
        SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
        FROM properties_2017
        LEFT JOIN propertylandusetype USING(propertylandusetypeid)
        WHERE propertylandusedesc IN ("Single Family Residential",                       
                                      "Inferred Single Family Residential")"""

        df = pd.read_sql(query, url)


        df = df.rename(columns = {'bedroomcnt':'bedrooms', 
                                  'bathroomcnt':'bathrooms', 
                                  'calculatedfinishedsquarefeet':'area',
                                  'taxvaluedollarcnt':'tax_value', 
                                  'yearbuilt':'year_built',})
        return df

In [2]:
#Importing all the things needed as well as the necessary columns for the data

In [3]:
zillow = acquire_zillow().dropna

In [4]:
#Replacing all null data with 0 to look less awkward

In [5]:
zillow_new = zillow().reset_index()

In [6]:
zillow_new

,bedrooms,bathrooms,square_feet,tax_value,year_built,taxamount,fips
0,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
1,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
2,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
3,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
4,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
...,...,...,...,...,...,...,...
2140230,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0
2140231,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2140232,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2140233,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0


In [7]:
df = zillow_new.astype('int64')

In [8]:
df

,bedrooms,bathrooms,square_feet,tax_value,year_built,taxamount,fips
0,4,2,3633,296425,2005,6941,6037
1,3,4,1620,847770,2011,10244,6037
2,3,2,2077,646760,1926,7924,6037
3,0,0,1200,5328,1972,91,6037
4,0,0,171,6920,1973,255,6037
...,...,...,...,...,...,...,...
2140230,4,4,4375,422400,2015,13877,6037
2140231,4,3,2262,960756,2015,13494,6059
2140232,4,4,3127,536061,2014,6244,6059
2140233,3,2,1974,424353,2015,5302,6059


In [9]:
df_sample = df.head(1000)

In [21]:
def bathroomtax(df): 
    plt.figure(figsize=(16,8))
    sns.barplot(data=df_sample,x='bathrooms',y='tax_value')
    plt.show()

In [22]:
def bedroomstax(df):
    plt.figure(figsize=(16,8))
    sns.barplot(data=df_sample,x='bedrooms',y='tax_value')
    plt.show()

In [24]:
def squaretax(df):
    plt.figure(figsize=(16,8))
    sns.scatterplot(data=df_sample,x='square_feet',y='tax_value')
    plt.show()

In [25]:
def squarerooms(df):
    plt.figure(figsize=(16,8))
    sns.scatterplot(data=df_sample,x='square_feet',y='bedrooms')
    plt.show()

In [26]:
def bedbathsq(df):
    plt.figure(figsize=(16,8))
    sns.lineplot(data=df_sample,x='bedrooms',y='tax_value')
    sns.lineplot(data=df_sample,x='bathrooms',y='tax_value')
    plt.legend(labels=["Bed rooms","Bed Divengence","Bath rooms","Bath Divengence"])
    plt.show()

In [27]:
def pairsquare(df):
    sns.pairplot(df_sample)
    plt.show()

In [28]:
def taxguess(df):
    plt.figure(figsize=(16,8))
    plt.hist(y_train.tax_value,label="Total Value")
    plt.hist(y_train.tv_pred_mean, bins=1,label="Predicted Tax value")
    plt.xlabel("Final value")
    plt.ylabel("Total")
    plt.legend()
    plt.show()

In [29]:
def predict(df):
    plt.figure(figsize=(16,8))

    plt.plot(y_validate.tax_value, y_validate.tv_pred_mean, alpha=.5, color="gray", label='_nolegend_')
    plt.annotate("Baseline", (16, 9.5))

    plt.plot(y_validate.tax_value, y_validate.tax_value, alpha=.5, color="blue", label='_nolegend_')
    plt.annotate("Ideal Line", (.5, 3.5), rotation=15.5)

    plt.scatter(y_validate.tax_value, y_validate.tv_pred_mean, 
            alpha=.5, color="red", s=100, label="Linear Regression")

    plt.scatter(y_validate.tax_value, y_validate.tv_pred_median, 
            alpha=.5, color="purple", s=100, label="Lasso Lars")

    plt.scatter(y_validate.tax_value, y_validate.tax_value, 
            alpha=.5, color="yellow", s=100, label="Tweedie Regressor")

    plt.scatter(y_validate.tax_value, y_validate.tax_value, 
            alpha=.5, color="green", s=100, label="Polynomial")
    plt.legend()
    plt.xlabel("Actual Total")
    plt.ylabel("Predicted Total")
    plt.title("Predictions most extreme?")
    plt.show()

In [31]:
def predictionofsome(df):
    plt.figure(figsize=(16,8))

    plt.hist(y_validate.tax_value, color='blue', alpha=.5, label="Actual Total")
    plt.hist(y_validate.tax_value, color='red', alpha=.5, label="Linear Regression")
    plt.hist(y_validate.tax_value, color='purple', alpha=.5, label="Lasso Lars")

    plt.xlabel("Total")
    plt.ylabel("Tax value")
    plt.title("Comparing the Distribution")
    plt.legend()
    plt.show()